In [1]:
import nltk
import re
import collections
import itertools
import lda
import requests
import csv
import time
import math
import operator
import numpy as np
import pandas as pd
from collections import defaultdict
from pandas import read_table
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [2]:
data_list = pd.read_csv("./data/감성분석/전처리_pos.csv", header=0)
data_list = data_list.drop("Unnamed: 0", axis=1)

In [3]:
data_list = data_list['doc']

In [4]:
data_list[0:10]

0    return a320 flight hour flight meaning alcohol...
1    wife speak food service provided china souther...
2    return late july august family enjoyable fligh...
3    boeing 737-800. seated bulkhead seat service w...
4    curious nervous try airline due review read on...
5    flight joy generous legroom meal fantastic ran...
6    original ticket booked economy class upgraded ...
7    considering ba fare high begin purchase seat s...
8    first time flew lufthansa sao paulo frankfurt ...
9    traveled dxb-mct-lhr-mct missed flight told se...
Name: doc, dtype: object

In [5]:
# 영어 숫자 특수기호 남기기
pre_list = []
for i in data_list:
    i = str(i)
    text = re.sub('[^a-zA-Z0-9]',' ',i).strip()
    text = re.sub('[,]','',text)
    text = re.sub('  ','',text)
    if(text != ''):
        if(text[0] !='?'):
            pre_list.append(text)

In [6]:
pre_list

['return a320 flight hour flight meaning alcoholic beverage cheese cracker vegetable crisp served bag christchurch flight biscuit vegetable crisp served tea coffee water bag minute seat room good tall issue flight lasting hour direction seated good value flight duration',
 'wife speak food service provided china southern traveled auckland guangzhou london return flight outstanding cabin crew went way make flight enjoyable food service exceptional seating bed first class comfortable bag make trip home arrive next flight limited inflight entertainment option offer compared western airline cabin service food overall value experience make',
 'return late july august family enjoyable flight aircraft clean appeared new legroom economy fantastic surprising crew attentive pleasant deal issue lack indonesian style meal available overall happy',
 'boeing 737 800seated bulkhead seat service water run completed guava juice followed offer pillow blanket arrival card newspaper offered choice main me

In [7]:
stop_words = 'flight','airline','class','time','service','hour','plane','return','very','however','time','bit','as','my','then','never','next','no','due','one','us','ife','entertainment','airplane','arrive','back','airport','business','economy','even','use','air','friend','experience','new','offer','aircraft','way','trip','go','bag','ticket','home','film','june','warm','bkk','jfk','a320','a319','cx','kl','icn','july','not','only','so','first','more','other','flight.','just','again','much','also','really','well','better'
nnp_list = []

for i in pre_list:
    origin_words = nltk.word_tokenize(i)
    data_pos = nltk.pos_tag(origin_words)
    words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR']]
    words_nnp = [w for w in words_nnp if not w in stopwords.words('english')]
    words_nnp = [w for w in words_nnp if not w in stop_words]
    nnp_list.append(words_nnp)

In [8]:
nnp_list_1d = list(itertools.chain.from_iterable(nnp_list))
nnp_list_1d[0:20]

['meaning',
 'alcoholic',
 'beverage',
 'cheese',
 'cracker',
 'vegetable',
 'crisp',
 'christchurch',
 'biscuit',
 'vegetable',
 'crisp',
 'tea',
 'coffee',
 'water',
 'minute',
 'seat',
 'room',
 'good',
 'tall',
 'issue']

In [9]:
words_list=nnp_list_1d

In [10]:
# # 토픽 모델링 과정.
# # WordCount
# word_count = collections.Counter(words_list)
# result = word_count.most_common(1000)
# print(len(word_count))
# print(result)

In [11]:
# # Countervoctorzier & LAD
# c_vetorizer = CountVectorizer(analyzer='word')
# count = c_vetorizer.fit_transform(words_list)

# model = lda.LDA(n_topics = 10, n_iter = 2000, random_state = 1)
# model.fit(count)

In [12]:
# # LDA result
# topic_vocab = c_vetorizer.get_feature_names()
# topic_word = model.topic_word_
# n_top_word = 100
# dist = []
# for i, topic_dist in enumerate(topic_word):
#     dist.append(topic_dist)
#     topic_words = np.array(topic_vocab)[np.argsort(topic_dist)][:-n_top_word:-1]
#     print('Topic', i+1, topic_words)

In [13]:
# # 데이터 파일 불러오기
# df=pd.read_csv(r"./data/감성분석/review_neg.csv",header=0,encoding="utf8",error_bad_lines=False)
# df.head()

In [2]:
import re
import numpy as np
import json
import nltk
import itertools
import collections
import lda
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from collections import defaultdict

In [15]:
# # 리뷰 데이터만 따로 지정
# data_list=df['doc']
# data_list

word_tokenize를 통해 단어 토큰화를 수행

In [16]:
# # 영어 숫자 특수기호 남기기
# review_list = []
# for i in data_list:
#     i = str(i)
#     i = i.lower()
#     text = re.sub('[^a-zA-Z0-9]',' ',i).strip()
#     text = re.sub('[,]','',text)
#     if(text != ''):
#         if(text[0] !='?'):
#             review_list.append(text)

In [17]:
# review_list

In [18]:
# review_list_df=pd.DataFrame(review_list)
# review_list_df.to_csv("./data/review_list.csv", encoding="utf-8")

불용어를 제거

In [3]:
review_list=pd.read_csv("./data/review_list.csv", encoding="utf-8")

In [5]:
review_list = list(review_list['doc'])

In [6]:
review_list

['auckland to christchurch return on a320  flight to christchurch koru hour flight meaning alcoholic beverages and cheese and cracker or vegetable crisps served  bags out quickly at christchurch  flight to auckland biscuits or vegetable crisps served along with tea coffee or water  bags out after 25 minutes  seat room pretty good  however for the very tall  could be issue  flight fast lasting about an hour in both directions  seated in 23f and 23a respectively  good value flight for the duration',
 'my wife and i cannot speak highly enough about the food and service provided by china southern  we recently traveled auckland guangzhou london return  all flights were outstanding  cabin crew went out of their way to make our flights enjoyable  the food and service was exceptional  seating and beds in first class were very comfortable  unfortunately my bag did not make it on to the plane in london for our trip home but did arrive on the next flight  although there is limited inflight entert

In [7]:
airport_names=pd.read_csv(r"./data/stopwords_airport.csv",header=0,encoding="utf8",error_bad_lines=False)
airport_names.head()

,airport
0,auckland
1,christchurch
2,london
3,guangzhou
4,singapore


공항이름 제거

In [8]:
stop2=list(airport_names['airport'])

In [9]:
stop2

['auckland',
 'christchurch',
 'london',
 'guangzhou',
 'singapore',
 'paris',
 'dubai',
 'melbourne',
 'denpasar',
 'sydney',
 'brisbane',
 'myrtle beach',
 'laguardia',
 'jfk',
 'tel aviv',
 'kiev',
 'london gatwick',
 'florence',
 'athens',
 'larnaca',
 'toron',
 'lisbon',
 'lax',
 'hefei',
 'beijing',
 'cincinnati',
 'phoenix',
 'manchester',
 'jfk new york',
 'vie',
 'los angeles',
 'milan',
 'lhr',
 'vienna',
 'geneva',
 'fort lauderdale',
 'fra',
 'clermont-ferrand',
 'newark',
 'hong kong',
 'new york',
 'auh',
 'raleigh-durham',
 'denver',
 'berlin',
 'dar es salaam',
 'ams',
 'jakarta',
 'bengkulu',
 'zagreb',
 'warsaw',
 'rome',
 'sao paulo',
 'frankfurt',
 'dxb',
 'mct',
 'kigali',
 'doha',
 'bangkok',
 'amsterdam',
 'amman',
 'johannesburg',
 'hog',
 'yul',
 'phuket',
 'london stansted',
 'paphos',
 'ankara',
 'istanbul',
 'm찼laga',
 'tokyo',
 'bos',
 'idaho falls',
 'hanoi',
 'atlanta',
 'montreal',
 'bangalore',
 'delhi',
 'clearwater st pete fl',
 'syracuse ny',
 'st. t

In [10]:
stop_words = 'flight','airline','class','time','service','hour','plane','return','very','however','time','bit','as','my','then','never','next','no','due','one','us','ife','entertainment','airplane','arrive','back','airport','business','economy','even','use','air','friend','experience','new','offer','aircraft','way','trip','go','bag','ticket','home','film','june','warm','bkk','jfk','a320','a319','cx','kl','icn','july','not','only','so','first','more','other','flight.','just','again','much','also','really','well','better'
nnp_list = []

for i in review_list:
    origin_words = nltk.word_tokenize(i)
    data_pos = nltk.pos_tag(origin_words)
    words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR']]
    words_nnp = [w for w in words_nnp if not w in stopwords.words('english')]
    words_nnp = [w for w in words_nnp if not w in stop_words]
    words_nnp = [w for w in words_nnp if not w in stop2]
    nnp_list.append(words_nnp)

In [11]:
print(nnp_list[0:5])

[['meaning', 'alcoholic', 'cheese', 'cracker', 'vegetable', 'crisps', 'vegetable', 'crisps', 'tea', 'coffee', 'water', 'seat', 'room', 'good', 'tall', 'lasting', 'good', 'value', 'duration'], ['wife', 'enough', 'food', 'china', 'southern', 'outstanding', 'cabin', 'food', 'exceptional', 'seating', 'comfortable', 'limited', 'inflight', 'cabin', 'food', 'overall', 'value'], ['late', 'early', 'august', 'family', 'enjoyable', 'clean', 'legroom', 'fantastic', 'surprising', 'crew', 'attentive', 'pleasant', 'issue', 'lack', 'indonesian', 'style', 'available', 'overall', 'happy'], ['boeing', 'bulkhead', 'seat', 'onboard', 'water', 'run', 'guava', 'juice', 'arrival', 'main', 'meal', 'ice', 'cream', 'lindt', 'chocolate', 'arrival', 'attentive', 'zealand', 'zealand', 'staff', 'job'], ['curious', 'nervous', 'online', 'glad', 'inflight', 'excellent', 'destination', 'top', 'opportunity', 'stringent', 'regarding', 'luggage', 'advice', 'pack', 'light']]


In [12]:
nnp_list_1d = list(itertools.chain.from_iterable(nnp_list))
nnp_list_1d[0:20]

['meaning',
 'alcoholic',
 'cheese',
 'cracker',
 'vegetable',
 'crisps',
 'vegetable',
 'crisps',
 'tea',
 'coffee',
 'water',
 'seat',
 'room',
 'good',
 'tall',
 'lasting',
 'good',
 'value',
 'duration',
 'wife']

표제어 추출

In [13]:
tokenized_doc=[]
for i in nnp_list_1d:
    a=WordNetLemmatizer().lemmatize(i)
    tokenized_doc.append(a)

In [14]:
tokenized_doc[0:20]

['meaning',
 'alcoholic',
 'cheese',
 'cracker',
 'vegetable',
 'crisp',
 'vegetable',
 'crisp',
 'tea',
 'coffee',
 'water',
 'seat',
 'room',
 'good',
 'tall',
 'lasting',
 'good',
 'value',
 'duration',
 'wife']

In [15]:
words_list=tokenized_doc

In [ ]:
# # 부정 리뷰 토픽 모델링

# data_list = pd.read_csv("./data/감성분석/전처리_neg.csv", header=0)
# data_list = data_list.drop("Unnamed: 0", axis=1)

In [ ]:
# data_list = data_list['doc']

In [ ]:
# data_list[0:10]

In [ ]:
# # 영어 숫자 특수기호 남기기
# pre_list = []
# for i in data_list:
#     i = str(i)
#     text = re.sub('[^a-zA-Z0-9]',' ',i).strip()
#     text = re.sub('[,]','',text)
#     text = re.sub('  ','',text)
#     if(text != ''):
#         if(text[0] !='?'):
#             pre_list.append(text)

In [ ]:
# pre_list

In [ ]:
# stop_words = 'flight','airline','class','time','service','hour','plane','return','very','however','time','bit','as','my','then','never','next','no','due','one','us','ife','entertainment','airplane','arrive','back','airport','business','economy','even','use','air','friend','experience','new','offer','aircraft','way','trip','go','bag','ticket','home','film','june','warm','bkk','jfk','a320','a319','cx','kl','icn','july','not','only','so','first','more','other','flight.','just','again','much','also','really','well','better'
# nnp_list = []

# for i in pre_list:
#     origin_words = nltk.word_tokenize(i)
#     data_pos = nltk.pos_tag(origin_words)
#     words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR']]
#     words_nnp = [w for w in words_nnp if not w in stopwords.words('english')]
#     words_nnp = [w for w in words_nnp if not w in stop_words]
#     nnp_list.append(words_nnp)

In [ ]:
# nnp_list_1d = list(itertools.chain.from_iterable(nnp_list))
# nnp_list_1d[0:20]

In [ ]:
# words_list=nnp_list_1d

In [ ]:
# # 토픽 모델링 과정.
# # WordCount
# word_count = collections.Counter(words_list)
# result = word_count.most_common(1000)
# print(len(word_count))
# print(result)

In [ ]:
# # Countervoctorzier & LAD
# c_vetorizer = CountVectorizer(analyzer='word')
# count = c_vetorizer.fit_transform(words_list)

# model = lda.LDA(n_topics = 8, n_iter = 2000, random_state = 1)
# model.fit(count)

In [ ]:
# # LDA result
# topic_vocab = c_vetorizer.get_feature_names()
# topic_word = model.topic_word_
# n_top_word = 100
# dist = []
# for i, topic_dist in enumerate(topic_word):
#     dist.append(topic_dist)
#     topic_words = np.array(topic_vocab)[np.argsort(topic_dist)][:-n_top_word:-1]
#     print('Topic', i+1, topic_words)

In [ ]:
#공항 불용어
airport_names=pd.read_csv("./data/stopwords_airport_2.csv")

In [ ]:
stop2=list(airport_names['airport'])

In [ ]:
stop2

In [16]:
# 나이브 베이지 과정
# 나이브 베이지안 모델
class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def count_words(self, training_set):
        # 학습데이터는 리뷰 본문(doc), 라벨(label)로 구성
        # 나이브 베이지안 stopwords = 1이상의 값을 가지는 단어와 불필요 상위 단어.
        stop_words = 'flight','airline','class','time','service','hour','plane','return','very','however','time','bit','as','my','then','never','next','no','due','one','us','ife','entertainment','airplane','arrive','back','airport','business','economy','even','use','air','friend','experience','new','offer','aircraft','way','trip','go','bag','ticket','home','film','june','warm','bkk','jfk','a320','a319','cx','kl','icn','july','not','only','so','first','more','other','flight.','just','again','much','also','really','well','better','boeing','lhr','b777'
        counts = defaultdict(lambda : [0, 0])
        for doc, label in training_set:
            # 리뷰가 text일 때만 카운트
            if self.isNumber(doc) is False:
                # 리뷰를 띄어쓰기 단위로 토크나이징
                words = doc.split()                           
                # 토픽 모델링과 같은 전처리.
                data_pos = nltk.pos_tag(words)
                words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR','RB','RBS','RBR']]
                words = [w for w in words_nnp if not w in stopwords.words('english')]
                words = [w for w in words_nnp if not w in stop2]
                
                for word in words:
                    text = str(word)
                    i = text.lower()
                    if(i not in stop_words and i not in stop2):
                        # 라벨이 1이면 0값 지정
                        counts[i][0 if label == 1 else 1] += 1
        return counts
    # 예외 처리
    def isNumber(self, s):
        try:
            float(s)
            return True
        except ValueError:
            return False
    # 모델 결과 반환.
    def word_probabilities(self, counts, total_class0, total_class1, k):
        # 단어의 빈도수를 [단어, p(w|긍정), p(w|부정)] 형태로 반환
        return [(w,
                 (class0 + k) / (total_class0 + 2*k),
                 (class1 + k) / (total_class1 + 2*k))
                for w, (class0, class1) in counts.items()]

    def class0_probability(self, word_probs, doc):
        # input 띄어쓰기 처리
        docwords = doc.split()
        # 초기값은 모두 0으로 처리
        log_prob_if_class0 = log_prob_if_class1 = 0.0

        for word, prob_if_class0, prob_if_class1 in word_probs:
            
            # 만약 리뷰에 word가 나타나면 해당 단어가 나올 log 확률을 더해 줌
            if word in docwords:
                log_prob_if_class0 += math.log(prob_if_class0)
                log_prob_if_class1 += math.log(prob_if_class1)
            # 만약 리뷰에 word가 나타나지 않는다면 해당 단어가 나오지 않을 log 확률을 더해 줌. 
            # 나오지 않을 확률은 log(1-나올 확률)로 계산
            else:
                log_prob_if_class0 += math.log(1.0 - prob_if_class0)
                log_prob_if_class1 += math.log(1.0 - prob_if_class1)

        prob_if_class0 = math.exp(log_prob_if_class0)
        prob_if_class1 = math.exp(log_prob_if_class1)
        return prob_if_class0 / (prob_if_class0 + prob_if_class1)
    # 모델 학습
    def train(self, corpus):
        training_set = corpus

        # calss0 = 긍정리뷰 수 / class1 = 부정리뷰 수 
        num_class0 = 18232
        num_class1 = 15357

        # train
        word_counts = self.count_words(training_set)
        self.word_probs = self.word_probabilities(word_counts, num_class0, num_class1, self.k)
    # 모델 테스트 
    def classify(self, doc):
        return self.class0_probability(self.word_probs, doc)

In [17]:
# 긍정 리뷰 불러오기
trainfile_path=r'./data/감성분석/review_pos.csv'
corpus = read_table(trainfile_path, sep=',', encoding='utf-8')
train_po = corpus[0:18232]
# train_po = train_po.drop("Unnamed: 0", axis=1)
corpus_po = np.array(train_po)

In [18]:
train_po

,doc,label
0,Auckland to Christchurch return on A320. Fligh...,0
1,My wife and I cannot speak highly enough about...,0
2,Melbourne to Denpasar return in late July / ea...,0
3,Brisbane to Auckland on Boeing 737-800. Seated...,0
4,I was curious and nervous to try this airline ...,0
...,...,...
18227,Singapore Changi Airport to Harbin via Shangha...,0
18228,NRT-SYD in Premium Economy. This cabin is a we...,0
18229,ORD-AMM-RUH in economy. The first flight was 1...,0
18230,BKK-HEL return. Flights punctual. New business...,0


In [19]:
# 부정 리뷰 불러오기
trainfile_path=r'./data/감성분석/review_neg.csv'
corpus = read_table(trainfile_path, sep=',', encoding='utf-8')
train_ne = corpus[0:15357]
# train_ne = train_ne.drop("Unnamed: 0", axis=1)
corpus_ne = np.array(train_ne)

In [20]:
# 긍정 부정 리뷰 병합 베이지안 모델 input
nb_data = np.concatenate((corpus_po, corpus_ne))

In [21]:
# 모델 학습
model = NaiveBayesClassifier()
model.train(nb_data)

In [22]:
# 긍정 단어 가중치로 딕셔너리 만들기
dict_p = {}
for i in model.word_probs:
    dict_p[i[0]] = i[1]
# 부정 단어 가중치로 딕셔너리 만들기    
dict_n = {}
for i in model.word_probs:
    dict_n[i[0]] = i[2]

In [23]:
# 각 딕셔너리 값을 내림차순 정렬
sort_dict_p = sorted(dict_p.items(), key=operator.itemgetter(1), reverse=True)
sort_dict_n = sorted(dict_n.items(), key=operator.itemgetter(1), reverse=True)

In [24]:
# 각 리뷰별 상위 단어 추출
for i in sort_dict_p[0:30]:
    print(i[0])

seat
food
staff
crew
cabin
boarding
good
last
flying
long
gate
poor
customer
same
old
leg
meal
late
luggage
being
extra
finally
worst
most
check
day
delay
still
again.
going


In [25]:
for i in sort_dict_n[0:30]:
    print(i[0])

good
food
seat
crew
staff
cabin
comfortable
great
friendly
excellent
leg
boarding
meal
lounge
flew
long
overall
time.
quite
flying
clean
check
check-in
short
inflight
little
full
always
most
old


In [85]:
# # 나이브 베이지안 모델 테스트
# a = 'Auckland to Christchurch return on A320. Flight to Christchurch Koru Hour flight meaning alcoholic beverages and Cheese and cracker or vegetable crisps served. Bags out quickly at Christchurch. Flight to Auckland biscuits or vegetable crisps served along with tea/coffee or water. Bags out after 25 minutes. Seat room pretty good. However for the very tall, could be issue. Flight fast lasting about an hour in both directions. Seated in 23F and 23A respectively. Good value flight for the duration.'
# a = a.lower()
# model.classify(a)

2.6122052368766826e-06

In [26]:
pos=pd.read_csv("./data/감성분석/pos_형용사_csv.csv", header=0, encoding='utf-8')
neg=pd.read_csv("./data/감성분석/neg_형용사_csv.csv", header=0, encoding='utf-8')

In [27]:
pos=list(pos['1'])
neg=list(neg['1'])

In [28]:
pos

['good',
 'professional',
 'special',
 'comfortable',
 'excellent',
 'helpful',
 'great',
 'friendly',
 'efficient',
 'early',
 'attentive',
 'pleasant',
 'quick',
 'extra',
 'polite',
 'positive',
 'recommend',
 'best',
 'smooth',
 'courteous',
 'fine',
 'standard',
 'apologetic',
 'wait',
 'superb',
 'smile',
 'impressed',
 'perfect',
 'informative',
 'happy',
 'abundance',
 'interested',
 'acceptable',
 'delightful',
 'calm',
 'welcome',
 'loyal',
 'nice',
 'beneficial',
 'reasonable',
 'brilliant',
 'surprised',
 'fast',
 'decent',
 'bright',
 'clean',
 'fantastic',
 'favourable',
 'enjoyable',
 'admirable',
 'responsive',
 'impressive',
 'ample',
 'premium',
 'assist',
 'enjoyed',
 'approachable',
 'appropriate',
 'helpful',
 'comfortable',
 'technical',
 'quiet',
 'best',
 'arrive',
 'informative',
 'courteous',
 'happy',
 'attentive',
 'professional',
 'acceptable',
 'exceptional',
 'supportive',
 'commendable',
 'wonderful',
 'communicative',
 'earlier',
 'professional',
 'spec

In [29]:
topic_pos=pd.read_csv("./data/감성분석/topic_pos.csv", header=0, encoding='utf-8')

In [30]:
#감성 사전 구축 과정
# 긍정 토픽
ps_comment = pos
# ps_meal = list(topic['meal'])
# ps_delay = list(topic['delay'])
# ps_cabin_staff = list(topic['cabin_staff'])
# ps_seat = list(topic['seat'])
# ps_ground_service = list(topic['ground_service'])
ps_meal = list(topic_pos['기내식'])
ps_cabin_staff = list(topic_pos['승무원'])
ps_seat = list(topic_pos['좌석'])

In [32]:
topic_neg=pd.read_csv("./data/감성분석/topic_neg.csv", header=0, encoding='utf-8')

In [33]:
# 부정 토픽
ne_comment = neg
# ne_meal = list(topic['meal'])
# ne_delay = list(topic['delay'])
# ne_cabin_staff = list(topic['cabin_staff'])
# ne_seat = list(topic['seat'])
# ne_ground_service = list(topic['ground_service'])
ne_delay = list(topic_neg['서비스지연'])
ne_ground_service = list(topic_neg['지상서비스'])

In [34]:
import numpy as np
import math
# Nan값을 리스트에서 제거
j = -1
for i in range(len(ne_delay)):
    j += 1
    if not isinstance(ne_delay[j], str) and math.isnan(ne_delay[j]):
        del ne_delay[j]
        j = j-1

In [35]:
j = -1
for i in range(len(ne_ground_service)):
    j += 1
    if not isinstance(ne_ground_service[j], str) and math.isnan(ne_ground_service[j]):
        del ne_ground_service[j]    
        j = j-1

In [36]:
j = -1
for i in range(len(ps_meal)):
    j += 1
    if not isinstance(ps_meal[j], str) and math.isnan(ps_meal[j]):
        del ps_meal[j]
        j = j-1

In [37]:
j = -1
for i in range(len(ps_cabin_staff)):
    j += 1
    if not isinstance(ps_cabin_staff[j], str) and math.isnan(ps_cabin_staff[j]):
        del ps_cabin_staff[j]
        j = j-1

In [38]:
j = -1
for i in range(len(ps_seat)):
    j += 1
    if not isinstance(ps_seat[j], str) and math.isnan(ps_seat[j]):
        del ps_seat[j]
        j = j-1

In [47]:
# 토픽 모델링 단어에 대해 나이브베이지안 확률값 매핑
ps_comment_dic = {}
for topic in ps_comment:
    for word,value in dict_p.items():
        if(topic == word):
            ps_comment_dic[word] = value
print(len(ps_comment_dic))

ps_meal_dic = {}
ne_meal_dic = {}
for topic in ps_meal:
    for word,value in dict_p.items():
        if(topic == word):
            ps_meal_dic[word] = value
print(len(ps_meal_dic))
for topic in ps_meal:
    for word,value in dict_n.items():
        if(topic == word):
            ne_meal_dic[word] = value
print(len(ne_meal_dic))
         
ps_cabin_staff_dic = {}
ne_cabin_staff_dic = {}
for topic in ps_cabin_staff:
    for word,value in dict_p.items():
        if(topic == word):
            ps_cabin_staff_dic[word] = value
print(len(ps_cabin_staff_dic))
for topic in ps_cabin_staff:
    for word,value in dict_n.items():
        if(topic == word):
            ne_cabin_staff_dic[word] = value
print(len(ne_cabin_staff_dic))

ps_seat_dic = {}
ne_seat_dic = {}
for topic in ps_seat:
    for word,value in dict_p.items():
        if(topic == word):
            ps_seat_dic[word] = value
print(len(ps_seat_dic))
for topic in ps_seat:
    for word,value in dict_n.items():
        if(topic == word):
            ne_seat_dic[word] = value
print(len(ne_seat_dic))
            
ne_comment_dic = {}
for topic in ne_comment:
    for word,value in dict_n.items():
        if(topic == word):
            ne_comment_dic[word] = value
print(len(ne_comment_dic))            

ne_delay_dic = {}
ps_delay_dic = {}
for topic in ne_delay:
    for word,value in dict_n.items():
        if(topic == word):
            ne_delay_dic[word] = value
print(len(ne_delay_dic))
for topic in ne_delay:
    for word,value in dict_p.items():
        if(topic == word):
            ps_delay_dic[word] = value
print(len(ps_delay_dic))

ne_ground_service_dic = {}
ps_ground_service_dic = {}
for topic in ne_ground_service:
    for word,value in dict_n.items():
        if(topic == word):
            ne_ground_service_dic[word] = value
print(len(ne_ground_service_dic))
ps_ground_service_dic = {}
for topic in ne_ground_service:
    for word,value in dict_p.items():
        if(topic == word):
            ps_ground_service_dic[word] = value
print(len(ps_ground_service_dic))

98
22
22
6
6
17
17
56
17
17
12
12


In [40]:
ps_meal_dic

{'food': 0.336121318488455,
 'wine': 0.027834146876542534,
 'snack': 0.01691987056436132,
 'breakfast': 0.037212746119673124,
 'water': 0.07395930455767016,
 'sandwich': 0.013135523501343718,
 'meal': 0.11339329786650579,
 'drink': 0.04039379147699226,
 'coffee': 0.02618877858827401,
 'dinner': 0.02701146273240827,
 'meals': 0.0009049525585476882,
 'chicken': 0.015548730324137552,
 'glass': 0.021088136894641583,
 'bottle': 0.013245214720561619,
 'lunch': 0.012148302528382603,
 'vegetarian': 0.013848516426260078,
 'juice': 0.010009323753633522,
 'tea': 0.01587780398179126,
 'quality': 0.04450721219766358,
 'bar': 0.008199418636538145,
 'catering': 0.008802720342236604,
 'alcohol': 0.006499204738660671}

In [46]:
ne_meal_dic

{'food': 0.6296067196249512,
 'wine': 0.057071233233493945,
 'snack': 0.05446672743846855,
 'breakfast': 0.07484698528454226,
 'water': 0.05668055736424014,
 'sandwich': 0.030244823544732388,
 'meal': 0.1685440812605808,
 'drink': 0.058438598775882276,
 'coffee': 0.03167730173199635,
 'dinner': 0.048476364109910144,
 'meals': 0.0038742023701002735,
 'chicken': 0.027640317749706992,
 'glass': 0.014227112905326215,
 'bottle': 0.013836437036072406,
 'lunch': 0.02718452923557755,
 'vegetarian': 0.015789816382341452,
 'juice': 0.01845943482224248,
 'tea': 0.0192407865607501,
 'quality': 0.07758171636931892,
 'bar': 0.02138950384164605,
 'catering': 0.02614272691756739,
 'alcohol': 0.01429222555020185}

In [48]:
with open('meal_dic.csv','w') as f:
    w = csv.writer(f)
    w.writerow(ps_meal_dic.keys())
    w.writerow(ps_meal_dic.values())
    w.writerow(ne_meal_dic.keys())
    w.writerow(ne_meal_dic.values())
    
with open('cabin_staff_dic.csv','w') as f:
    w = csv.writer(f)
    w.writerow(ps_cabin_staff_dic.keys())
    w.writerow(ps_cabin_staff_dic.values())
    w.writerow(ne_cabin_staff_dic.keys())
    w.writerow(ne_cabin_staff_dic.values())
    
with open('seat_dic.csv','w') as f:
    w = csv.writer(f)
    w.writerow(ps_seat_dic.keys())
    w.writerow(ps_seat_dic.values())
    w.writerow(ne_seat_dic.keys())
    w.writerow(ne_seat_dic.values())
    
with open('delay_dic.csv','w') as f:
    w = csv.writer(f)
    w.writerow(ne_delay_dic.keys())
    w.writerow(ne_delay_dic.values())
    w.writerow(ps_delay_dic.keys())
    w.writerow(ps_delay_dic.values())
    
with open('ground_service_dic.csv','w') as f:
    w = csv.writer(f)
    w.writerow(ne_ground_service_dic.keys())
    w.writerow(ne_ground_service_dic.values())
    w.writerow(ps_ground_service_dic.keys())
    w.writerow(ps_ground_service_dic.values())
    
with open('ps_comment_dic.csv','w') as f:
    w = csv.writer(f)
    w.writerow(ps_comment_dic.keys())
    w.writerow(ps_comment_dic.values())
    
with open('ne_comment_dic.csv','w') as f:
    w = csv.writer(f)
    w.writerow(ne_comment_dic.keys())
    w.writerow(ne_comment_dic.values())

In [50]:
# 딕셔너리 매핑 확인
print(ps_comment_dic)

{'good': 0.15447265946360994, 'professional': 0.008747874732627654, 'special': 0.017303789831623978, 'comfortable': 0.043465145615093514, 'excellent': 0.01867493007184775, 'helpful': 0.020375143969725222, 'great': 0.044781440245708334, 'friendly': 0.04467174902649043, 'efficient': 0.010502934240114078, 'early': 0.03584160587944935, 'attentive': 0.009351176438326112, 'pleasant': 0.019278231777546208, 'quick': 0.008473646684582899, 'extra': 0.10834750178248231, 'polite': 0.010996544726594636, 'positive': 0.01264191301486316, 'recommend': 0.0009049525585476882, 'best': 0.035896451489058304, 'smooth': 0.006060439861789064, 'courteous': 0.004853836450392146, 'fine': 0.028437448582240992, 'standard': 0.02284319640212801, 'apologetic': 0.002605166456425163, 'wait': 0.021691438600340042, 'superb': 0.0009597981681566391, 'smile': 0.00918663960949926, 'impressed': 0.002934240114078868, 'perfect': 0.002605166456425163, 'informative': 0.00030165085284922943, 'happy': 0.014451818131958536, 'abundan

In [51]:
print(ps_seat_dic)

{'seat': 0.3369988482421982, 'cabin': 0.19851368397959743, 'legroom': 0.03468984807766139, 'row': 0.04812702243185433, 'leg': 0.11657434322382493, 'choice': 0.05004661876816761, 'space': 0.03798058465419843, 'room': 0.0770306586957714, 'seating': 0.04757856633576482, 'screen': 0.032660560522130205, 'lounge': 0.05180167827565403, 'sleep': 0.009570558876761915, 'flat': 0.01757801787966873, 'sector': 0.008583337903800801, 'rest': 0.024049799813524928, 'interior': 0.007212197663577031, 'bed': 0.006937969615532277}


In [23]:
print(ps_comment_dic)

NameError: name 'ps_comment_dic' is not defined

In [61]:
comment_dic={**ps_comment_dic, **ne_comment_dic}

In [81]:
# def positvie_review(doc):
#     doc = doc
#     comment = 0
#     meal = 0
#     cabin_staff = 0
#     seat = 0
#     total = 0
    
#     words = doc.split()                           
#     data_pos = nltk.pos_tag(words)
#     words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR','RB','RBS','RBR']]
#     words = [w for w in words_nnp if not w in stopwords.words('english')]
    
#     # 각 토픽별 점수 계산
#     for word in words:
#         for _,i in ps_comment_dic.items():
#             if (word == _):
#                 comment += i
#         for _,i in ps_meal_dic.items():
#             if (word == _):
#                 meal += i
#         for _,i in ps_cabin_staff_dic.items():
#             if (word == _):
#                 cabin_staff += i
#         for _,i in ps_seat_dic.items():
#             if (word == _):
#                 seat += i
                
#     total = comment + meal + cabin_staff + seat
    
        
#     print('------------------------------------------------------리뷰 감성 분석------------------------------------------------------')
#     print('-------------------------------------------------------토픽별 점수--------------------------------------------------------')
#     print('리뷰평  : '+str(comment)+'  | 기내식 : '+str(meal)+'  | 승무원 : '+str(cabin_staff)+'  | 좌석 : '+str(seat))
#     print('총 합계 : '+str(total))
#     print('--------------------------------------------------------------------------------------------------------------------------')

In [62]:
# def positvie_review1(doc):
#     doc = doc
#     comment = 0
#     meal = 0
#     cabin_staff = 0
#     seat = 0
#     final_list=[]
#     total = 0
    
#     words = doc.split()                           
#     data_pos = nltk.pos_tag(words)
#     words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR','RB','RBS','RBR']]
#     words = [w for w in words_nnp if not w in stopwords.words('english')]
    
#     # 각 토픽별 점수 계산
#     for word in words:
#         for _,i in ps_comment_dic.items():
#             if (word == _):
#                 comment += i
#         for _,i in ps_meal_dic.items():
#             if (word == _):
#                 meal += i
#         for _,i in ps_cabin_staff_dic.items():
#             if (word == _):
#                 cabin_staff += i
#         for _,i in ps_seat_dic.items():
#             if (word == _):
#                 seat += i
                
#     total = comment + meal + cabin_staff + seat
#     # 리뷰를 최대값을 갖는 토픽으로 할당
#     if max(meal, cabin_staff, seat) == meal:
#         final_list.append('meal')
#         final_list.append(total)
#         return final_list
    
#     elif max(meal, cabin_staff, seat) == cabin_staff:
#         final_list.append('cabin_staff')
#         final_list.append(total)
#         return final_list
    
#     elif max(meal, cabin_staff, seat) == seat:
#         final_list.append('seat')
#         final_list.append(total)
#         return final_list
        
# #     print('------------------------------------------------------리뷰 감성 분석------------------------------------------------------')
# #     print('-------------------------------------------------------토픽별 점수--------------------------------------------------------')
# #     print('리뷰평  : '+str(comment)+'  | 기내식 : '+str(meal)+'  | 승무원 : '+str(cabin_staff)+'  | 좌석 : '+str(seat))
# #     print('총 합계 : '+str(total))
# #     print('--------------------------------------------------------------------------------------------------------------------------')

In [82]:
# def negative_review(doc):
#     doc = doc
#     comment = 0
#     delay = 0
#     ground_service = 0
#     total = 0

#     words = doc.split()                           
#     data_pos = nltk.pos_tag(words)
#     words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR','RB','RBS','RBR']]
#     words = [w for w in words_nnp if not w in stopwords.words('english')]
    
#     # 각 토픽별 점수 계산
#     for word in words:
#         for _,i in ne_comment_dic.items():
#             if (word == _):
#                 comment += i
        
#         for _,i in ne_delay_dic.items():
#             if (word == _):
#                 delay += i
        
#         for _,i in ne_ground_service_dic.items():
#             if (word == _):
#                 ground_service += i
        
#     total = comment + delay + ground_service
    
    
#     print('------------------------------------------------------리뷰 감성 분석------------------------------------------------------')
#     print('-------------------------------------------------------토픽별 점수--------------------------------------------------------')
#     print('리뷰평  : '+str(comment)+'  | 서비스 지연 : '+str(delay)+'  | 지상서비스 : '+str(ground_service))
#     print('총 합계 : '+str(total))
#     print('--------------------------------------------------------------------------------------------------------------------------')

In [66]:
# def negative_review1(doc):
#     doc = doc
#     comment = 0
#     delay = 0
#     ground_service = 0
#     final_list=[]
#     total = 0

#     words = doc.split()                           
#     data_pos = nltk.pos_tag(words)
#     words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR','RB','RBS','RBR']]
#     words = [w for w in words_nnp if not w in stopwords.words('english')]
    
#     # 각 토픽별 점수 계산
#     for word in words:
#         for _,i in ne_comment_dic.items():
#             if (word == _):
#                 comment += i
        
#         for _,i in ne_delay_dic.items():
#             if (word == _):
#                 delay += i
        
#         for _,i in ne_ground_service_dic.items():
#             if (word == _):
#                 ground_service += i
        
#     total = comment + delay + ground_service
    
#     # 리뷰를 최대값을 갖는 토픽으로 할당
#     if max(delay, ground_service) == delay:
#         final_list.append('delay')
#         final_list.append(total)
#         return final_list
    
#     elif max(delay, ground_service) == ground_service:
#         final_list.append('ground_service')
#         final_list.append(total)
#         return final_list
        
# #     print('------------------------------------------------------리뷰 감성 분석------------------------------------------------------')
# #     print('-------------------------------------------------------토픽별 점수--------------------------------------------------------')
# #     print('리뷰평  : '+str(comment)+'  | 서비스 지연 : '+str(delay)+'  | 지상서비스 : '+str(ground_service))
# #     print('총 합계 : '+str(total))
# #     print('--------------------------------------------------------------------------------------------------------------------------')

In [66]:
def review1(doc):
    doc = doc
    comment = 0
    
    meal = 0
    cabin_staff = 0
    seat = 0
    delay = 0
    ground_service = 0
    
    final_list=[]
    ps_total = 0
    ne_total = 0
    
    words = doc.split()                           
    data_pos = nltk.pos_tag(words)
    words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR','RB','RBS','RBR']]
    words = [w for w in words_nnp if not w in stopwords.words('english')]
    
    # 각 토픽별 점수 계산
    for word in words:
        for _,i in comment_dic.items():
            if (word == _):
                comment += i
        for _,i in ps_meal_dic.items():
            if (word == _):
                meal += i
        for _,i in ps_cabin_staff_dic.items():
            if (word == _):
                cabin_staff += i
        for _,i in ps_seat_dic.items():
            if (word == _):
                seat += i
        
        for _,i in comment_dic.items():
            if (word == _):
                comment += i
        for _,i in ne_comment_dic.items():
            if (word == _):
                comment += i
        
        for _,i in ne_delay_dic.items():
            if (word == _):
                delay += i
        
        for _,i in ne_ground_service_dic.items():
            if (word == _):
                ground_service += i
                
    po_total = comment + meal + cabin_staff + seat
    ne_total = comment + delay + ground_service
    # 리뷰를 최대값을 갖는 토픽으로 할당
    if max(meal, cabin_staff, seat,delay, ground_service) == meal:
        final_list.append('meal')
        final_list.append(ps_total)
        return final_list
    
    elif max(meal, cabin_staff, seat,delay, ground_service) == cabin_staff:
        final_list.append('cabin_staff')
        final_list.append(ps_total)
        return final_list
    
    elif max(meal, cabin_staff, seat,delay, ground_service) == seat:
        final_list.append('seat')
        final_list.append(ps_total)
        return final_list
    
    elif max(meal, cabin_staff, seat,delay, ground_service) == delay:
        final_list.append('delay')
        final_list.append(ne_total)
        return final_list
    
    elif max(meal, cabin_staff, seat,delay, ground_service) == ground_service:
        final_list.append('ground_service')
        final_list.append(ne_total)
        return final_list
        
#     print('------------------------------------------------------리뷰 감성 분석------------------------------------------------------')
#     print('-------------------------------------------------------토픽별 점수--------------------------------------------------------')
#     print('리뷰평  : '+str(comment)+'  | 기내식 : '+str(meal)+'  | 승무원 : '+str(cabin_staff)+'  | 좌석 : '+str(seat))
#     print('총 합계 : '+str(total))
#     print('--------------------------------------------------------------------------------------------------------------------------')

In [76]:
def review2(doc):
    doc = doc
    comment = 0
    
    meal = 0
    cabin_staff = 0
    seat = 0
    delay = 0
    ground_service = 0
    
    final_list=[]
    ps_total = 0
    ne_total = 0
    
    words = doc.split()                           
    data_pos = nltk.pos_tag(words)
    words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR','RB','RBS','RBR']]
    words = [w for w in words_nnp if not w in stopwords.words('english')]
    
    # 각 토픽별 점수 계산
    for word in words:
        for _,i in ps_comment_dic.items():
            if (word == _):
                comment += i
        for _,i in ps_meal_dic.items():
            if (word == _):
                meal += i
        for _,i in ps_cabin_staff_dic.items():
            if (word == _):
                cabin_staff += i
        for _,i in ps_seat_dic.items():
            if (word == _):
                seat += i
        
        for _,i in comment_dic.items():
            if (word == _):
                comment += i
        for _,i in ne_comment_dic.items():
            if (word == _):
                comment += i
        
        for _,i in ne_delay_dic.items():
            if (word == _):
                delay += i
        
        for _,i in ne_ground_service_dic.items():
            if (word == _):
                ground_service += i
                
    po_total = comment + meal + cabin_staff + seat + delay + ground_service
    
            
    print('------------------------------------------------------리뷰 감성 분석------------------------------------------------------')
    print('-------------------------------------------------------토픽별 점수--------------------------------------------------------')
    print('리뷰평  : '+str(comment)+'  | 기내식 : '+str(meal)+'  | 승무원 : '+str(cabin_staff)+'  | 좌석 : '+str(seat)+
          '  | 서비스 지연 : '+str(delay)+'  | 지상서비스 : '+str(ground_service))
    print('총 합계 : '+str(po_total))
    print('--------------------------------------------------------------------------------------------------------------------------')

In [103]:
# 모델 평가
po = """Excellent comfort and food, which is served on demand during the flight.
The seat converts into a flat bed at the press of a button. The seat and the pillow was ultra-comfortable.
Passengers are given a set of pyjamas for red-eye flight and amenities from Giorgio Armani in a pouch.
Service, however, can be improved as it is not consistent. I had to wave to get the attention of the flight attendant as no one had attended to me for more than 5 min after the attendant light was turned on. I also had to remind the flight steward to collect my glass prior to take-off. No one reminded me to stow away my tray table prior to landing."""

ne = """My family flew to Fort Lauderdale and back to Concord on separate flights.
My first flight left on time but my sister's flight was scheduled to leave at 9am did not leave until 3.45 pm.
On return flight my plane was scheduled to take off at 6 pm, we finally got on the plane after 7 pm and didn't take off until 8.30pm.
My sister's return flight was scheduled to take off at 1 pm and didn't take off until 3.30. The customer service is the worst I have experienced. I needed to change a reservation (hotel room) and waited over an hour and a half for a someone to pick up the phone.
Altogether I was on the phone for 2 hours and 24 minutes to change one hotel room."""

positvie_review(po)
negative_review(ne)

------------------------------------------------------리뷰 감성 분석------------------------------------------------------
-------------------------------------------------------토픽별 점수--------------------------------------------------------
리뷰평  : 0.011435309603466243  | 기내식 : 0.021088136894641583  | 승무원 : 0  | 좌석 : 0.6985136839795975
총 합계 : 0.7310371304777052
--------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------리뷰 감성 분석------------------------------------------------------
-------------------------------------------------------토픽별 점수--------------------------------------------------------
리뷰평  : 0.011427269175673917  | 서비스 지연 : 0.02340799583279073  | 지상서비스 : 0
총 합계 : 0.03483526500846465
--------------------------------------------------------------------------------------------------------------------------


In [16]:
po=list(train_po['doc'])
po

['Auckland to Christchurch return on A320. Flight to Christchurch Koru Hour flight meaning alcoholic beverages and Cheese and cracker or vegetable crisps served. Bags out quickly at Christchurch. Flight to Auckland biscuits or vegetable crisps served along with tea/coffee or water. Bags out after 25 minutes. Seat room pretty good. However for the very tall, could be issue. Flight fast lasting about an hour in both directions. Seated in 23F and 23A respectively. Good value flight for the duration.',
 'My wife and I cannot speak highly enough about the food and service provided by China Southern. We recently traveled Auckland-Guangzhou-London return. All flights were outstanding. Cabin crew went out of their way to make our flights enjoyable. The food and service was exceptional. Seating and beds in First Class were very comfortable. Unfortunately my bag did not make it on to the plane in London for our trip home but did arrive on the next flight. Although there is limited inflight enter

In [67]:
pos_review_classify=[]


po=list(train_po['doc'])

for i in po:
    pos_review_classify.append(review1(i))


In [68]:
neg_review_classify=[]
ne=list(train_ne['doc'])
for j in ne:
    neg_review_classify.append(review1(j))

In [69]:
pos_review_classify

[['seat', 0],
 ['meal', 0],
 ['cabin_staff', 0],
 ['ground_service', 0.42900968456046523],
 ['cabin_staff', 0],
 ['meal', 0],
 ['cabin_staff', 0],
 ['cabin_staff', 0],
 ['cabin_staff', 0],
 ['cabin_staff', 0],
 ['seat', 0],
 ['meal', 0],
 ['seat', 0],
 ['cabin_staff', 0],
 ['seat', 0],
 ['meal', 0],
 ['meal', 0],
 ['seat', 0],
 ['seat', 0],
 ['ground_service', 2.3547259773997946],
 ['ground_service', 1.1689561857716149],
 ['ground_service', 1.4606098888926797],
 ['meal', 0],
 ['seat', 0],
 ['seat', 0],
 ['seat', 0],
 ['meal', 0],
 ['cabin_staff', 0],
 ['ground_service', 0.5538662326509334],
 ['ground_service', 1.0012957087785124],
 ['seat', 0],
 ['seat', 0],
 ['cabin_staff', 0],
 ['meal', 0],
 ['ground_service', 0.9658933570224846],
 ['meal', 0],
 ['seat', 0],
 ['seat', 0],
 ['seat', 0],
 ['meal', 0],
 ['meal', 0],
 ['ground_service', 0.8846702446469161],
 ['ground_service', 1.8243427435062392],
 ['cabin_staff', 0],
 ['ground_service', 0.7192526506110328],
 ['ground_service', 0.2829903

In [70]:
ppp=pd.DataFrame(pos_review_classify)
ppp

,0,1
0,seat,0.00000
1,meal,0.00000
2,cabin_staff,0.00000
3,ground_service,0.42901
4,cabin_staff,0.00000
...,...,...
18227,meal,0.00000
18228,seat,0.00000
18229,meal,0.00000
18230,seat,0.00000


In [71]:
ppp.to_csv("./data/감성분석/리뷰별 감성점수/pos.csv", header=0, encoding="utf-8")

In [72]:
nnn=pd.DataFrame(neg_review_classify)
nnn

,0,1
0,ground_service,0.780913
1,ground_service,1.992842
2,ground_service,0.495680
3,ground_service,0.667730
4,seat,0.000000
...,...,...
15352,meal,0.000000
15353,cabin_staff,0.000000
15354,meal,0.000000
15355,meal,0.000000


In [73]:
nnn.to_csv("./data/감성분석/리뷰별 감성점수/neg.csv", header=0, encoding="utf-8")

In [77]:
po = """Excellent comfort and food, which is served on demand during the flight.
The seat converts into a flat bed at the press of a button. The seat and the pillow was ultra-comfortable.
Passengers are given a set of pyjamas for red-eye flight and amenities from Giorgio Armani in a pouch.
Service, however, can be improved as it is not consistent. I had to wave to get the attention of the flight attendant as no one had attended to me for more than 5 min after the attendant light was turned on. I also had to remind the flight steward to collect my glass prior to take-off. No one reminded me to stow away my tray table prior to landing."""

ne = """My family flew to Fort Lauderdale and back to Concord on separate flights.
My first flight left on time but my sister's flight was scheduled to leave at 9am did not leave until 3.45 pm.
On return flight my plane was scheduled to take off at 6 pm, we finally got on the plane after 7 pm and didn't take off until 8.30pm.
My sister's return flight was scheduled to take off at 1 pm and didn't take off until 3.30. The customer service is the worst I have experienced. I needed to change a reservation (hotel room) and waited over an hour and a half for a someone to pick up the phone.
Altogether I was on the phone for 2 hours and 24 minutes to change one hotel room."""

review2(po)

------------------------------------------------------리뷰 감성 분석------------------------------------------------------
-------------------------------------------------------토픽별 점수--------------------------------------------------------
리뷰평  : 0.022870619206932487  | 기내식 : 0.021088136894641583  | 승무원 : 0  | 좌석 : 0.6985136839795975  | 서비스 지연 : 0  | 지상서비스 : 0
총 합계 : 0.7424724400811715
--------------------------------------------------------------------------------------------------------------------------
